In [185]:
!pip install sentencepiece
!pip install -U sentence-transformers


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [186]:
import numpy as np
from nltk.tokenize.treebank import TreebankWordDetokenizer
from transformers import pipeline, AutoTokenizer
from nltk.tokenize import wordpunct_tokenize
mask_filler = pipeline("fill-mask", 'distilbert-base-uncased')

In [187]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [188]:
import os, random
SEED = 0
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED)

#### Здесь описываются параметры и задается входное предложение:  

In [189]:
SENT_NUM = 10
TEMPERATURE = 0.05
TOP_K = 5
#input_sent = "After your workout, remember to focus on maintaining a good water balance."
input_sent = "The latest tech news about the world's best (and sometimes worst) hardware, apps, and much more."

In [190]:
# probability of using gpt2 to replace ending of sentence at the end
USE_GPT2_PROBA = 0.5

In [191]:
generator = pipeline('text-generation', model='gpt2')

In [192]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [193]:
import gc
embedding_input = model.encode(input_sent, convert_to_tensor=True)

In [194]:
WORDS_TO_REPLACE = len(wordpunct_tokenize(input_sent))//3

CROP_WORDS = min(len(wordpunct_tokenize(input_sent))//2, 5)
input_sent = input_sent.lower()
curr_res = input_sent
detokenizer = TreebankWordDetokenizer()
predicted_sentences = []
while len(predicted_sentences) < SENT_NUM:
    for _ in range(WORDS_TO_REPLACE):
        words = wordpunct_tokenize(curr_res)
        word_to_replace = np.random.randint(len(words))
        words[word_to_replace] = tokenizer.mask_token
        sequence = detokenizer.detokenize(words)
        prediction = mask_filler(sequence, top_k=1)[0]
        curr_res = prediction['sequence']
        if prediction == words[word_to_replace-1]:
            del words[word_to_replace]
            curr_res = detokenizer.detokenize(words)
    
        # with probability TEMPERATURE or while sentence not chnges take predict from [1 TOP_K) pos
        if np.random.rand() < TEMPERATURE:
            curr_res = mask_filler(sequence, top_k=TOP_K)[np.random.randint(1,TOP_K)]['sequence']
        while curr_res == input_sent:
            curr_res = mask_filler(sequence, top_k=TOP_K)[np.random.randint(1,TOP_K)]['sequence']
    if np.random.rand() < USE_GPT2_PROBA and len(words) > 5:
        # crop some part of sentence and continue it with gpt2:
        words = wordpunct_tokenize(curr_res)
        tmp_res = generator(detokenizer.detokenize(words[:-CROP_WORDS]), max_length=len(words) + CROP_WORDS*2, num_return_sequences=1)[0]['generated_text']
        tmp_res = tmp_res[:tmp_res.rfind('.') + 1] # crop by '.' from right end
        if len(tmp_res) > len(curr_res) and tmp_res.rfind('.')!=-1: # we suppose sentence ended if it ended with '.' now
            curr_res = tmp_res
    sent_similarity = util.pytorch_cos_sim(embedding_input, 
                            model.encode(curr_res, convert_to_tensor=True)
                            ).item()
    if sent_similarity < 0.65: # then regenerate:
        continue
    # print(sent_similarity)
    predicted_sentences.append(curr_res)
    
    curr_res = input_sent

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.9342713952064514
0.7986621856689453


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.8621166348457336
0.717326819896698
0.6796664595603943


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.6527443528175354
0.7529996633529663


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.7963331341743469


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.8939462304115295
0.8368752598762512


In [196]:
input_sent

"the latest tech news about the world's best (and sometimes worst) hardware, apps, and much more."

In [195]:
predicted_sentences

["publishes latest tech news covering the world's best ( and possibly worst ) hardware, apps, and much more.",
 "the latest tech blogs about the world's best ( and sometimes worst ) hardware, software, and possibly more.",
 "publishes latest technology news covering the world's fastest ( and sometimes worst ) hardware, apps, and even more.",
 "wired wired tech news covers the world's fastest ( and sometimes worst ) hardware, apps, and even more.",
 "the latest tech guide covers the world's best ( and sometimes worst ) websites, apps, and many more.",
 "its official blog talks about computing world' s best (and sometimes worst) hardware acceleration software...and about the new (but perhaps not so new!) Apple software.",
 "the latest latest article about the world's best ( and sometimes worst ) hardware security apps, reveals much more.",
 "the latest tech news about the world's best ( and sometimes worst ) hardware security apps, is much more.",
 "the latest tech news about the world's